## Amazon Bedrock Flow Generation 

This notebook shows how we can take the Amazon Bedrock Knowledge Bases we created in `setup_knowledge_bases.ipynb` and put them in a structured flow using Amazon Bedrock  Flows (https://aws.amazon.com/bedrock/prompt-flows/).

Aamzon Bedrock Flows accelerates the creation, testing, and deployment of workflows through an intuitive visual builder. Flows allows you to seamlessly link foundation models (FMs), prompts, and many AWS services and tools together. In this notebook we are going to define our flow programmatically using `boto3`, but, you will be able to test your flow in the UI.

Our flow will allow us to have a versioned application where we can specify all of the sequential components, as well as any conditions we want to model. 

We create prompts we will store in [Amazon Bedrock Prompt Management](https://aws.amazon.com/bedrock/prompt-management/). Bedrock's prompt management application allows us to create persistent and versioned prompts, which we will then use in our flow.

We will start with a description of a RAG framework with additional modules (e.g., current date, web search, etc.) to generate a flow as shown below.

** Generated Flow**

![alt text](prompt_flow_asset/example_pf_output.png "Automatically Generated Flow")

Let's start!

**Table of Contents:**

1. [Complete prerequisites](#Complete%20prerequisites)
    
    a. [Configure logging](#Configure%20logging)
            
    b. [Organize imports](#Organize%20imports)
    
    c. [Set AWS Region and boto3 config](#Set%20AWS%20Region%20and%20boto3%20config)
    
    d. [Create common objects](#Create%20common%20objects)
    
    e. [Get details of Knowledge Bases](#Get%20details%20of%20Knowledge%20Bases)

 2. [Create Prompts](#Load%20data%20to%20Knowledge%20Bases)
    
    a. [Step 0a: Create prompt for routing](#Load%20to%20KB%20Step0b)
    
    b. [Step 0b: Create prompt for AI Assistant](#Load%20to%20KB%20Steps0c%20to%200e)
 
 3. [Create Flow](#Process%20query)
 
     a. [Step 1: Create flow](#User%20query)
     
     b. [Step 2: Create flow version](#Determine%20the%20KB%20id)
     
     c. [Step 3: create flow alias](#Retrieve%20and%20generate)
 
 4. [Execute Flows](#Cleanup)

 5. [Use Flow in UI](#Cleanup)
 
 6. [Conclusion](#Conclusion)

####  b. Application logs <a id='Configure%20application%20logs'></a>

Application logs refers to the logs generated by running the various code cells in this notebook. To set this up, instantiate the [Python logging service](https://docs.python.org/3/library/logging.html) by running the following cell. You can configure the default log level and format as required.

By default, this notebook will only print the logs to the corresponding cell's output console.

In [ ]:
import logging
import os

# Set the logging level and format
log_level = logging.INFO
log_format = '%(asctime)s - %(levelname)s - %(message)s'
logging.basicConfig(level=log_level, format=log_format)

# Explicity set the AWS Region for Amazon Bedrock clients
AMAZON_BEDROCK_DEFAULT_REGION = "us-west-2"
br_region = os.environ.get('AMAZON_BEDROCK_REGION')
if br_region is None:
    br_region = AMAZON_BEDROCK_DEFAULT_REGION
elif len(br_region) == 0:
    br_region = AMAZON_BEDROCK_DEFAULT_REGION
logging.info("AWS Region for Amazon Bedrock: {}".format(br_region))

# Save these in the environment variables for use in the helper scripts
os.environ['LOG_LEVEL'] = str(log_level)
os.environ['LOG_FORMAT'] = log_format

###  B. Organize imports <a id ='Organize%20imports'> </a>


In [ ]:
import boto3
from datetime import datetime
import json
import sys
import os
import sagemaker

# Import the helper functions from the 'scripts' folder
sys.path.append(os.path.join(os.getcwd(), "scripts"))
#logging.info("Updated sys.path: {}".format(sys.path))
from helper_functions import *

role = sagemaker.get_execution_role()

bedrock_agent = boto3.client(service_name="bedrock-agent", region_name="us-west-2")
bedrock_agent_rt = boto3.client(service_name="bedrock-agent-runtime", region_name="us-west-2")


###  C. Get KB Details <a id ='Organize%20imports'> </a>

Let's retrieve our Knowledge Bases, we will need these when we create our flow.

In [ ]:
kb_1_name = 'rag-router-kb-1'
kb_2_name = 'rag-router-kb-2'


kb_1_id, kb_1_ds_id, kb_1_s3_bucket_name, kb_1_aoss_collection_arn = get_kb_details(bedrock_agent, kb_1_name)
kb_2_id, kb_2_ds_id, kb_2_s3_bucket_name, kb_2_aoss_collection_arn = get_kb_details(bedrock_agent, kb_2_name)

###  2. Create Prompts <a id =Load%20data%20to%20Knowledge%20Bases> </a>

Let's create a prompt for our AI Assistant. Here we will use Amazon's Prompt Management service, which allows us to store and version our prompts. We specify a few things in our prompt
- prompt template
- inference parameters
- input variables

In [ ]:
# Create the prompt
assistant_response = bedrock_agent.create_prompt(
    name=f"AIAssistantPrompt-{datetime.now().strftime('%Y%m%d-%H%M%S')}",
    description="AI Assistant prompt for Amazon Bedrock Knowledge Base",
    variants=[
        {
            "inferenceConfiguration": {
                "text": {
                    "maxTokens": 4096,
                    "temperature": 0,
                    # "topP": 0.1,
                }
            },
            "modelId": "anthropic.claude-3-haiku-20240307-v1:0",
            "name": "variant-001",
            "templateConfiguration": {
                "text": {
                    "inputVariables": [
                        {"name": "document"},
                        {"name": "retrievalResults"}
                    ],
                    "text": """You are an AI assistant who is well-versed in Amazon Bedrock Knowledge base designed to answer a user's question.
Please generate a code about Amazon Bedrock Knowledge Base only when a user asks for it.
Do NOT use your own knowledge as facts in answers. 
Please output a list in order of the most valuable data sources for a knowledge base when appropriate.
Don't restate the instructions.

User query: {{document}}

Knowledge base results: {{retrievalResults}}

Please provide a response based on the above information:"""
                }
            },
            "templateType": "TEXT"
        }
    ],
    defaultVariant="variant-001"
)

###  E. Create Prompts <a id =Load%20data%20to%20Knowledge%20Bases> </a>

Let's create a prompt for our AI Router. This prompt will let a LLM choose between which data source makes the most sense to answer a user's question.

In [ ]:
router_response = bedrock_agent.create_prompt(
    name=f"AIRouterPrompt-{datetime.now().strftime('%Y%m%d-%H%M%S')}",
    description="AI Router prompt for Amazon Bedrock Knowledge Base",
    variants=[
        {
            "inferenceConfiguration": {
                "text": {
                    "maxTokens": 500,
                    "temperature": 0,
                    # "topP": 0.1,
                }
            },
            "modelId": "anthropic.claude-3-haiku-20240307-v1:0",
            "name": "variant-001",
            "templateConfiguration": {
                "text": {
                    "inputVariables": [
                        {"name": "QUERY"}
                    ],
                    "text": """Carefully take a look at the CATEGORY information specified in the <KBs> tag.
<KBs>
<CATEGORY1>code</CATEGORY1>
<CATEGORY2>docs</CATEGORY2>
</KBs>

Look at the query specified in the <QUERY> tag.

<QUERY>
{{QUERY}}
</QUERY>

Now, look at the output JSON format specified in the <OUTPUT> tag.

<OUTPUT>
chosen_category
</OUTPUT>

Now, follow the instructions specified in the <INSTRUCTIONS> tag.
<INSTRUCTIONS>
- Identify the category for the query specified in the <QUERY> tag. It should be one of the values specified in the <CATEGORY> tags inside the <KBs> tag.
- Based on the identified category, create an output  message as specified in the <OUTPUT> tag with corresponding values for "category".
- Your response should ONLY be a valid output as specified in the <OUTPUT> tag.
- Do not make up an answer.
- Do not include any preamble or postamble.
</INSTRUCTIONS>"""
                }
            },
            "templateType": "TEXT"
        }
    ],
    defaultVariant="variant-001"
)

In [ ]:
assistant_prompt_arn = assistant_response["arn"]
print(f"Assistant Prompt ARN: {assistant_prompt_arn}")
router_prompt_arn = router_response["arn"]
print(f"Router Prompt ARN: {router_prompt_arn}")

Now to retrieve our prompt we can simply run the below boto3 command.

In [ ]:
bedrock_agent.get_prompt(promptIdentifier = assistant_prompt_arn)

###  3. Create Flow <a id =Load%20data%20to%20Knowledge%20Bases> </a>

Amazon Bedrock Flows allow users to create end to end workflows using Amazon Bedrock and services like Amazon Bedrock Knowledge Bases and Prompt Management. 

Flows consists of nodes and connections. Nodes can consist of LLM prompts, conditions, data sources, lambdas, or data output/storage. 

The flow definition we are creating below consists of an initial prompt that routes between two data sources, one being the Bedrock user guide, the other being a series of AWS blog posts about Bedrock Knowledge Bases with some code samples. Based on the output of the LLM call, a conditional node will send us to one of the aforementioned knowledge bases, and then complete with a final LLM call answering the question using the required context. 

Let's create a full flow.

In [ ]:
# Create the flow
def create_flow_definition(kb_1_id, kb_2_id, assistant_prompt_arn, router_prompt_arn):
    """Create a flow definition we will use to generate a flow"""
    return {'connections': 
            [
                {'configuration': 
                 {'data': 
                  {'sourceOutput': 'document',
         'targetInput': 'QUERY'}},
       'name': 'FlowInputNodeFlowInputNode0ToPrompt_1PromptsNode0',
       'source': 'FlowInputNode',
       'target': 'Prompt_1',
       'type': 'Data'},
      {'configuration': 
       {'data': 
        {'sourceOutput': 'modelCompletion',
         'targetInput': 'conditionInput'}},
       'name': 'Prompt_1PromptsNode0ToConditionNode_1ConditionNode0',
       'source': 'Prompt_1',
       'target': 'ConditionNode_1',
       'type': 'Data'},
      {'configuration': 
       {'data': 
        {'sourceOutput': 'modelCompletion',
         'targetInput': 'document'}},
       'name': 'Prompt_3PromptsNode0ToFlowOutputNodeFlowOutputNode0',
       'source': 'Prompt_3',
       'target': 'FlowOutputNode',
       'type': 'Data'},
      {'configuration': 
       {'data': 
        {'sourceOutput': 'modelCompletion',
         'targetInput': 'document'}},
       'name': 'Prompt_2PromptsNode0ToFlowOutputNode_1FlowOutputNode0',
       'source': 'Prompt_2',
       'target': 'FlowOutputNode_1',
       'type': 'Data'},
      {'configuration': 
       {'data': 
        {'sourceOutput': 'retrievalResults',
         'targetInput': 'retrievalResults'}},
       'name': 'KnowledgeBaseNode_1KnowledgeBaseNode0ToPrompt_3PromptsNode0',
       'source': 'KnowledgeBaseNode_1',
       'target': 'Prompt_3',
       'type': 'Data'},
      {'configuration': 
       {'data': 
        {'sourceOutput': 'retrievalResults',
         'targetInput': 'retrievalResults'}},
       'name': 'KnowledgeBaseNode_2KnowledgeBaseNode0ToPrompt_2PromptsNode0',
       'source': 'KnowledgeBaseNode_2',
       'target': 'Prompt_2',
       'type': 'Data'},
      {'configuration': 
       {'conditional': 
        {'condition': 'Condition'}},
       'name': 'ConditionNode_1ConditionNodeHandle0ToKnowledgeBaseNode_2KnowledgeBaseNode_2HeaderHandle',
       'source': 'ConditionNode_1',
       'target': 'KnowledgeBaseNode_2',
       'type': 'Conditional'},
      {'configuration': 
       {'conditional': 
        {'condition': 'default'}},
       'name': 'ConditionNode_1ConditionNodeHandleDefaultConditionNode_1ToKnowledgeBaseNode_1KnowledgeBaseNode_1HeaderHandle',
       'source': 'ConditionNode_1',
       'target': 'KnowledgeBaseNode_1',
       'type': 'Conditional'},
      {'configuration': 
       {'data': 
        {'sourceOutput': 'document',
         'targetInput': 'retrievalQuery'}},
       'name': 'FlowInputNodeFlowInputNode0ToKnowledgeBaseNode_2KnowledgeBaseNode0',
       'source': 'FlowInputNode',
       'target': 'KnowledgeBaseNode_2',
       'type': 'Data'},
      {'configuration': 
       {'data': 
        {'sourceOutput': 'document',
         'targetInput': 'retrievalQuery'}},
       'name': 'FlowInputNodeFlowInputNode0ToKnowledgeBaseNode_1KnowledgeBaseNode0',
       'source': 'FlowInputNode',
       'target': 'KnowledgeBaseNode_1',
       'type': 'Data'},
      {'configuration':
       {'data': 
        {'sourceOutput': 'document',
         'targetInput': 'document'}},
       'name': 'FlowInputNodeFlowInputNode0ToPrompt_2PromptsNode1',
       'source': 'FlowInputNode',
       'target': 'Prompt_2',
       'type': 'Data'},
      {'configuration': 
       {'data': 
        {'sourceOutput': 'document',
         'targetInput': 'document'}},
       'name': 'FlowInputNodeFlowInputNode0ToPrompt_3PromptsNode1',
       'source': 'FlowInputNode',
       'target': 'Prompt_3',
       'type': 'Data'}],
     'nodes': 
            [
                {'configuration': {'input': {}},
       'name': 'FlowInputNode',
       'outputs': [{'name': 'document', 'type': 'String'}],
       'type': 'Input'},
      {'configuration': 
       {'output': {}},
       'inputs': [{'expression': '$.data', 'name': 'document', 'type': 'String'}],
       'name': 'FlowOutputNode',
       'type': 'Output'},
      {'configuration': 
       {'condition': 
        {'conditions': [
            {'expression': 'conditionInput=="code"',
           'name': 'Condition'},
          {'name': 'default'}]}},
       'inputs': [{'expression': '$.data',
         'name': 'conditionInput',
         'type': 'String'}],
       'name': 'ConditionNode_1',
       'type': 'Condition'},
      {'configuration': 
       {'prompt': 
        {'sourceConfiguration': 
         {'resource': {'promptArn': router_prompt_arn}}}},
       'inputs': [{'expression': '$.data', 'name': 'QUERY', 'type': 'String'}],
       'name': 'Prompt_1',
       'outputs': [{'name': 'modelCompletion', 'type': 'String'}],
       'type': 'Prompt'},
      {'configuration': 
       {'output': {}},
       'inputs': [{'expression': '$.data', 'name': 'document', 'type': 'String'}],
       'name': 'FlowOutputNode_1',
       'type': 'Output'},
      {'configuration': 
       {'prompt': 
        {'sourceConfiguration': 
         {'resource': {'promptArn': assistant_prompt_arn}}}},
       'inputs': [{'expression': '$.data',
         'name': 'retrievalResults',
         'type': 'Array'},
        {'expression': '$.data', 'name': 'document', 'type': 'String'}],
       'name': 'Prompt_2',
       'outputs': [{'name': 'modelCompletion', 'type': 'String'}],
       'type': 'Prompt'},
      {'configuration': 
       {'prompt': 
        {'sourceConfiguration': 
         {'resource': {'promptArn': assistant_prompt_arn}}}},
       'inputs': [{'expression': '$.data',
         'name': 'retrievalResults',
         'type': 'Array'},
        {'expression': '$.data', 'name': 'document', 'type': 'String'}],
       'name': 'Prompt_3',
       'outputs': [{'name': 'modelCompletion', 'type': 'String'}],
       'type': 'Prompt'},
      {'configuration': 
       {'knowledgeBase': {'knowledgeBaseId': kb_1_id}},
       'inputs': [{'expression': '$.data',
         'name': 'retrievalQuery',
         'type': 'String'}],
       'name': 'KnowledgeBaseNode_1',
       'outputs': [{'name': 'retrievalResults', 'type': 'Array'}],
       'type': 'KnowledgeBase'},
      {'configuration': 
       {'knowledgeBase': {'knowledgeBaseId': kb_2_id}},
       'inputs': [{'expression': '$.data',
         'name': 'retrievalQuery',
         'type': 'String'}],
       'name': 'KnowledgeBaseNode_2',
       'outputs': [{'name': 'retrievalResults', 'type': 'Array'}],
       'type': 'KnowledgeBase'}]}


flow_definition1 = create_flow_definition(kb_1_id, kb_2_id, assistant_prompt_arn, router_prompt_arn)

response = bedrock_agent.create_flow(
    name=f"BedrockKnowledgeBaseFlow-{datetime.now().strftime('%Y%m%d-%H%M%S')}",
    description="Flow for answering questions about Amazon Bedrock Knowledge Base",
    executionRoleArn=role,
    definition=flow_definition1
)

flow_id = response["id"]
flow_arn = response["arn"]
flow_name = response["name"]
print(f"Flow ID: {flow_id}")
print(f"Flow ARN: {flow_arn}")
print(f"Flow Name: {flow_name}")

# Prepare the flow
response = bedrock_agent.prepare_flow(flowIdentifier=flow_id)
print(json.dumps(response, indent=2, default=str))

####  3. Create Flow Version <a id='Configure%20application%20logs'></a>

Once we have created our flow, we then can create a version of our flow we can then deploy. Let's create our version 1.

In [ ]:
version_response = bedrock_agent.create_flow_version(
    description='main-flow-version',
    flowIdentifier=flow_id
)
version_response

####  3. Create Flow Alias <a id='Configure%20application%20logs'></a>

Once we have created our flow version , we then can create an alias we can use to call our flow. Let's create one.

In [ ]:
alias_response = bedrock_agent.create_flow_alias(
    description='Main flow Alias',
    flowIdentifier=flow_id,
    name="main-flow-alias",
    routingConfiguration=[
        {
            'flowVersion': '1'
        },
    ],
)
alias_response

####  4. Execute Flows <a id='Configure%20application%20logs'></a>

Once we have created our flows, we can test them out! Let's try a test question. 

In [ ]:

query = "tell me about KBs"

response = bedrock_agent_rt.invoke_flow(
    flowAliasIdentifier=alias_response['id'],
    flowIdentifier=flow_id,
    inputs=[
        {
            'content': {
                'document': query
            },
            'nodeName': 'FlowInputNode',
            'nodeOutputName': 'document'
        },
    ]
)

flow_output1 = [response for response in iter(response['responseStream'])][0]['flowOutputEvent']['content']['document']
print(flow_output1)


Let's try one more question that should trigger our other knowledge base.

In [ ]:

query = "Give me code to create a knowledge base"

response = bedrock_agent_rt.invoke_flow(
    flowAliasIdentifier=alias_response['id'],
    flowIdentifier=flow_id,
    inputs=[
        {
            'content': {
                'document': query
            },
            'nodeName': 'FlowInputNode',
            'nodeOutputName': 'document'
        },
    ]
)

flow_output2 = [response for response in iter(response['responseStream'])][0]['flowOutputEvent']['content']['document']
print(flow_output2)

In [ ]:
# Print the Amazon Bedrock flow page URL
logging.info("Amazon Bedrock model access page - https://{}.console.aws.amazon.com/bedrock/home?region={}#/flows" 
             .format(br_region, br_region))

####  5. Use Flow in UI <a id='Configure%20application%20logs'></a>

After successfully executing the flow deployment code, your flow is deployed to your AWS account. 

You can use the above generated link to access your flow

Your flow will be in the flows list in the main panel. 

![alt text](prompt_flow_asset/example_pf_list.png "Flows List")


Click the generate flow and Click Edit in flow builder button in the top right corner.

![alt text](prompt_flow_asset/example_pf_panel.png "Flow Panel")

You will be able to see the draft of the generated flow and test it with your questions about Amazon Bedrock Knowledge Base.

![alt text](prompt_flow_asset/example_pf_output.png "Automatically Generated Flow")

####  6. Conclusion <a id='Configure%20application%20logs'></a>

Now we have seen how we can take our application and generate a Bedrock Flow using it. Let's now finish up with our 3rd and final notebook, `eval_flows.ipynb`